In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

Excluding at the moment all the columns that contains NaN (Data set to zero or -999)

In [4]:
def exclude_NaN(tX):
    sort_no_NaN= [1,2,3,7,8,9,10,11,13,14,15,16,17,18,19,20,21,28,29]
    tX_reduced = tX[:,sort_no_NaN]
    return tX_reduced
tX = exclude_NaN(tX)
tX.shape

(250000, 19)

Normalization of the data, to get 0 mean and unit variance

In [5]:
def normalize(tX):
    print(np.mean(tX))
normalize(tX)

3.69328881937


# Doing something with gradient descent first

Let us first try out with some simple least squares, which should give a bad solution

In [5]:
import gradient_descent as gd

Initial guess

In [19]:
N0=500
losses0,w = gd.gradient_descent_MAE(y,tX2,[0.5]*19,N,0.00005)
N=2000 
losses,w = gd.gradient_descent_MAE(y,tX2,w[N0-1],N,0.00001)

In [20]:
print('Losses :',losses[N-1])

Losses : 11.538227599


We want to compute the error made by our prediction with RMSE measurement

In [21]:
import costs as co
def RMSE(y,tX,weight):
    return np.sqrt(2*co.compute_loss(y,tX,weight))

In [22]:
weights = w[1000]

In [23]:
RMSE(y,tX2,weights)

7.7403783201110983

Now we will do prediction on our test data

## Generate predictions and save ouput in csv format for submission:

In [24]:
DATA_TEST_PATH = 'data/test.csv'  # TODO: download train data and supply path here 
y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
tX_test_sorted = exclude_NaN(tX_test)

In [25]:
OUTPUT_PATH = 'data/output.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test_sorted)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

# Trying out Ridge regression now

In [34]:
def split_data(x, y, ratio, seed=1):
    """ Split the dataset based on the split ratio.
        The order is : x_train, x_test,y_train,y_test
    """
    # set seed
    np.random.seed(seed)
    assert len(x)==len(y)
    p = np.random.permutation(len(x))
    x_train,x_test = np.split(x[p],[int(ratio*len(x))])
    y_train,y_test = np.split(y[p],[int(ratio*len(y))])
    return x_train,x_test,y_train,y_test

In [98]:
import ridge_regression as rr
import build_polynomial as bp
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-4, 2, 20)   
    weights_arr = np.zeros((len(lambdas),(degree+1)*tX2.shape[1]))
    rmse_tr = np.zeros(len(lambdas))
    rmse_te = np.zeros(len(lambdas))
    # ***************************************************
    # split the data, and return train and test data:
    # ***************************************************
    x_train,x_test,y_train,y_test = split_data(x,y,ratio,seed)
    
    # ***************************************************
    # form train and test data with polynomial basis function:
    # ***************************************************
    X_train = new_build_poly(x_train, degree)
    X_test = new_build_poly(x_test, degree)    
    
    # ***************************************************
    # ridge regression with different lambda: TODO
    # ***************************************************
    for i,lambd in enumerate(lambdas):
        #print(weights_arr[i,:].shape, ' ',X_train.shape)
        weights_arr[i,:] = rr.ridge_regression(y_train,X_train,lambd)
        rmse_tr[i] = np.sqrt(2*co.compute_loss(y_train,X_train,weights_arr[i,:]))
        rmse_te[i] = np.sqrt(2*co.compute_loss(y_test,X_test,weights_arr[i,:]))
                
        print("lambda ={l}, proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
                l=lambd,p=ratio, d=degree, tr=rmse_tr[i], te=rmse_te[i]))
    return lambdas, rmse_tr,rmse_te

seed = 6
degree = 2
split_ratio = 0.5
lambd, rmse_tr,rmse_te = ridge_regression_demo(tX2, y, degree, split_ratio, seed)

lambda =0.0001, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.00020691380811147902, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.00042813323987193956, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.0008858667904100823, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.0018329807108324356, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.00379269019073225, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.007847599703514606, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.01623776739188721, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.03359818286283781, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.06951927961775606, proportion=0.5, degree=2, Training RMSE=0.815, Testing RMSE=1.019
lambda =0.14384498882876628, proportion=0.5, degree=

Test for a build_poly function

In [99]:
def new_build_poly(x, degree):
    """polynomial basis function."""
    # Creates the matrix with the degrees we want to apply to our data x
    # replecating the vector with [1 2 3 4 ... degree] to a matrix
    #degree_mat = np.tile(list(range(0,degree+1)), (len(x), 1))
    #return np.transpose(np.power(x,np.transpose(degree_mat)))
    
    #Seulement 1 1, parce qu'il n'y a pas besoin de 30
    
    X=np.zeros((x.shape[0],(degree+1)*x.shape[1]))
    for i in range(degree+1):
        for j in range(x.shape[1]):
            #print(i*x.shape[1]+j)
            X[:,i*x.shape[1]+j]=x[:,j]**i
    return X